# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Fake News/"

In [0]:
import os
os.chdir('/content/drive/My Drive/Fake News/')

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:

import pandas
tbodies= pandas.read_csv(project_path+'train_bodies.csv')
tstances= pandas.read_csv(project_path+'train_stances.csv')
train = pandas.merge(tbodies, tstances.drop_duplicates(['Body ID']), on="Body ID", how="left") 


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [7]:
train.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,4,Last week we hinted at what was to come as Ebo...,It Begins: HazMat-Wearing Passenger Spotted At...,discuss
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...,"Comfort eating? Chinese woman, 26, spends an e...",unrelated
3,6,"Posting photos of a gun-toting child online, I...",‘The cub of Baghdadi': ISIS reports its younge...,discuss
4,7,At least 25 suspected Boko Haram insurgents we...,ISIS Beheads American Journalist,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [10]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer


Using TensorFlow backend.


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
comb_data = train['articleBody'] + train['Headline']
tokenizer.fit_on_texts(comb_data)

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
word_idx = tokenizer.word_index

In [0]:
from nltk.tokenize import sent_tokenize
texts = []
articles = []
for i in range(len(train['articleBody'])):
  texts.append(train['articleBody'].iloc[i])
  articles.append(sent_tokenize(train['articleBody'].iloc[i]))

## Check 2:

first element of texts and articles should be as given below. 

In [15]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [16]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence
import numpy as np

In [18]:
data = np.zeros((len(articles), MAX_SENTS, MAX_SENT_LENGTH))
data.shape

(1683, 20, 20)

In [19]:
for i, art in enumerate(articles):
  for j, sent in enumerate(art):
    print('processing sentence ....', j)
    if j < MAX_SENTS:
      dummy = text_to_word_sequence(sent)
      for k, word in enumerate(dummy):
        if k < MAX_SENT_LENGTH:
          data[i][j][k] = dict(tokenizer.word_index)[word]

processing sentence .... 0
processing sentence .... 1
processing sentence .... 2
processing sentence .... 3
processing sentence .... 4
processing sentence .... 5
processing sentence .... 6
processing sentence .... 7
processing sentence .... 8
processing sentence .... 9
processing sentence .... 10
processing sentence .... 11
processing sentence .... 12
processing sentence .... 13
processing sentence .... 14
processing sentence .... 15
processing sentence .... 0
processing sentence .... 1
processing sentence .... 2
processing sentence .... 3
processing sentence .... 4
processing sentence .... 0
processing sentence .... 1
processing sentence .... 2
processing sentence .... 3
processing sentence .... 4
processing sentence .... 5
processing sentence .... 6
processing sentence .... 7
processing sentence .... 8
processing sentence .... 9
processing sentence .... 10
processing sentence .... 11
processing sentence .... 0
processing sentence .... 1
processing sentence .... 2
processing sentence 

KeyError: ignored

### Check 3:

Accessing first element in data should give something like given below.

In [20]:
data[0, :, :]

array([[3.0000e+00, 4.0900e+02, 4.4600e+02, 6.3600e+03, 8.2000e+01,
        3.0000e+00, 3.6210e+03, 3.1600e+02, 5.0000e+00, 3.9200e+03,
        3.7900e+02, 4.0000e+00, 1.4630e+03, 2.9570e+03, 1.0000e+00,
        1.0200e+02, 1.5000e+01, 4.2300e+02, 0.0000e+00, 0.0000e+00],
       [8.3600e+02, 9.3000e+01, 1.3660e+03, 3.0000e+00, 2.3950e+03,
        2.3960e+03, 7.0000e+00, 1.7600e+02, 3.0000e+00, 9.4800e+02,
        1.4150e+03, 1.8870e+03, 7.2800e+02, 1.8800e+02, 1.0000e+00,
        3.5080e+03, 4.1000e+02, 1.0000e+00, 6.7700e+02, 2.6700e+02],
       [1.0200e+02, 1.0900e+03, 4.0760e+03, 2.6420e+03, 1.5000e+01,
        3.0000e+00, 1.2300e+03, 3.2580e+03, 1.9000e+01, 1.0000e+00,
        1.0200e+02, 2.0000e+00, 1.9340e+03, 1.0000e+00, 5.2200e+02,
        2.6430e+03, 1.4000e+01, 1.0000e+01, 3.0000e+00, 3.0590e+03],
       [1.9200e+02, 3.3810e+03, 8.5500e+02, 1.6400e+02, 2.3520e+03,
        4.3000e+01, 5.5460e+03, 1.8440e+03, 1.0440e+03, 5.0000e+00,
        1.0631e+04, 1.6669e+04, 1.0000e+00, 6

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
texts_heading = []
articles_heading = []
for i in range(len(train['Headline'])):
  texts_heading.append(train['Headline'].iloc[i])
  articles_heading.append(sent_tokenize(train['Headline'].iloc[i]))

In [22]:
data1 = np.zeros((len(articles_heading), MAX_SENTS, MAX_SENT_LENGTH))
data1.shape

(1683, 20, 20)

In [23]:
for i, art in enumerate(articles_heading):
  for j, sent in enumerate(art):
    print('processing sentence ....', j)
    if j < MAX_SENTS:
      dummy = text_to_word_sequence(sent)
      for k, word in enumerate(dummy):
        if k < MAX_SENT_LENGTH:
          data1[i][j][k] = dict(tokenizer.word_index)[word]

processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 1
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 1
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
processing sentence .... 0
p

KeyError: ignored

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
import pandas as pd
labels = pd.get_dummies(train['Stance'])

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [28]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (1683, 20, 20)
Shape of label tensor: (1683, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data1 = data1[indices]

In [0]:
## shuffle the labels according to data
labels = labels.iloc[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val = train_test_split(data, test_size=0.20, random_state=42)
x_heading_train, x_heading_val = train_test_split(data1, test_size=0.20, random_state=42)
y_train, y_val = train_test_split(labels, test_size=0.20, random_state=42)

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [33]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(1346, 20, 20)
(1346, 4)
(337, 20, 20)
(337, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [34]:
vocab_size = 400000
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam
from keras import backend
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape

### Model

In [0]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam
from keras import backend
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape

In [42]:
# Define the Type of Model
    model1 = Sequential()

    # Flatten Imgaes to Vector
    model1.add(Dense(1346, input_shape=(1346, 20, 20)))

    # Layer 1
    model1.add(Dense(output_dim=128, init='he_normal', bias=True)) # init='he_normal' -- 
    model1.add(Activation("relu"))

    # Layer 2
    model1.add(Dense(output_dim=3, init='he_normal', bias=True))
    model1.add(Activation("softmax"))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=128, kernel_initializer="he_normal", use_bias=True)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=3, kernel_initializer="he_normal", use_bias=True)`
  # This is added back by InteractiveShellApp.init_path()


### Compile and fit the model

In [43]:
# Loss and Optimizer
    model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # Train the model
    model1.fit(x_train, y_train, nb_epoch=10, batch_size=500)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: ignored

## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)